In [1]:
#%pip install dropbox

In [2]:
#%pip install tqdm

In [3]:
import dropbox
from dropbox import DropboxOAuth2FlowNoRedirect

import os
from tqdm import tqdm

In [4]:
APP_KEY = ''
APP_SECRET = ''

In [5]:
auth_flow = DropboxOAuth2FlowNoRedirect(APP_KEY, APP_SECRET)

In [6]:
authorize_url = auth_flow.start()

In [7]:
authorize_url

'https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=8i9m6sgxepa8f8o'

In [13]:
auth_code = input('Enter code here').strip()

In [14]:
try:
    oauth_result = auth_flow.finish(auth_code)
except Exception as e:
    print('Error: %s' % (e, ))
    exit(1)

In [18]:
file_from = '/home/studio-lab-user/SartoriusCellSegmentation/models/modelfull50eposcycle.pth'
file_to = '/modelfull50eposcycle.pth'

In [19]:
def upload(
    access_token,
    file_path,
    target_path,
    timeout=900,
    chunk_size=4 * 1024 * 1024,
):
    dbx = dropbox.Dropbox(access_token, timeout=timeout)
    with open(file_path, "rb") as f:
        file_size = os.path.getsize(file_path)
        chunk_size = 4 * 1024 * 1024
        if file_size <= chunk_size:
            print(dbx.files_upload(f.read(), target_path))
        else:
            with tqdm(total=file_size, desc="Uploaded") as pbar:
                upload_session_start_result = dbx.files_upload_session_start(
                    f.read(chunk_size)
                )
                pbar.update(chunk_size)
                cursor = dropbox.files.UploadSessionCursor(
                    session_id=upload_session_start_result.session_id,
                    offset=f.tell(),
                )
                commit = dropbox.files.CommitInfo(path=target_path)
                while f.tell() < file_size:
                    if (file_size - f.tell()) <= chunk_size:
                        print(
                            dbx.files_upload_session_finish(
                                f.read(chunk_size), cursor, commit
                            )
                        )
                    else:
                        dbx.files_upload_session_append(
                            f.read(chunk_size),
                            cursor.session_id,
                            cursor.offset,
                        )
                        cursor.offset = f.tell()
                    pbar.update(chunk_size)

In [20]:
upload(oauth_result.access_token, file_from, file_to)

Uploaded: 352321536it [02:19, 2522619.36it/s]                               

FileMetadata(client_modified=datetime.datetime(2021, 12, 21, 17, 35, 46), content_hash='507bb5f893ae4512cc641fe2ac6436df19858df8ce0dd69eed61ca5adf015fae', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:H0kgas51ERAAAAAAAAAVkQ', is_downloadable=True, media_info=NOT_SET, name='modelfull50eposcycle.pth', parent_shared_folder_id=NOT_SET, path_display='/modelfull50eposcycle.pth', path_lower='/modelfull50eposcycle.pth', property_groups=NOT_SET, rev='015d3ab6f7702da00000002556acd50', server_modified=datetime.datetime(2021, 12, 21, 17, 35, 46), sharing_info=NOT_SET, size=351103347, symlink_info=NOT_SET)
